# Importazione

## Importazione pacchetti

In [3]:
import pandas as pd
import json
import re
import spacy
import numpy as np
from collections import Counter
from functools import reduce
#from  tqdm import tqdm
import math
from IPython.display import display, HTML
nlp = spacy.load('en_core_web_sm')
from nltk.stem import SnowballStemmer
import json

## Import data

In [4]:
df=pd.read_csv("Dati/dati_puliti.csv")
df.drop(columns={'Unnamed: 0'},inplace=True)

DF2=pd.read_csv("Dati/tfidf_index.csv", index_col=None)
DF2.drop(columns={'Unnamed: 0'},inplace=True)

## Import vocabulary

In [5]:
with open("Dati/vocabulary.json", 'r') as v:
    vocabulary = json.load(v)
    
with open("Dati/vocabulary.json", 'r') as wd:
    word_dict  = json.load(wd)
    
with open("Dati/inverted_idx.json", 'r') as inv_idx:
    inverted_idx = json.load(inv_idx)

# Engine

## Stemmer

In [6]:
def stem_text(txt):
    stemmer_snowball = SnowballStemmer('italian')
    doc = nlp(txt.lower())
    return [re.sub("[\"”“/}/(/)/{!?\]\[#@]*\\n","",stemmer_snowball.stem(token.lemma_)) for token in doc if not token.is_punct and token.pos_ not in ['DET', 'CCONJ']]

df['list_words'] =df.title_trama.apply(lambda row: stem_text(row))

## Ranking engine all Match
Un documento deve contenere tutte le parole presenti nella query per essere ritrovato

### Funzioni

In [23]:
def querty_td_idf_allMatch(query_str):
    #Inizialize dataset
    td_inf={}
    
    #Remove double spaces
    query_str=re.sub("\s{2,}"," ",query_str)
    
    #Stem the query
    stemmed_query=stem_text(query_str)
    
    
    word_in_query=set(stemmed_query)
    
    
    for word in word_in_query:
    
        
        #DF document frequency
        df_t=vocabulary[word]
        if df_t==0:
            raise Exception("Can't find a document that contain all the word in the query, check input for misspelled words.")
           # raise Value Error("Can't find a document that contain all the word in the query, check input for misspelled words.")
            #print("Can't find a document that contain all the word in the query, check input for misspelled words.")
            #return None
        else:
    
            #TF
            ##occurence of t word in the query
            t_in_document=" ".join(stemmed_query).count(word) #count on total query or stemmed query? for now stemmed query


            ##Word in  the query
            parole_in_d=len(stemmed_query) #count on stemmed query or total query?for now stemmed query

            tf=t_in_document/parole_in_d
            
            #DF document frequency
            #df_t=vocabulary[word]
            results = [inverted_idx[str(item)] for item in [word_dict[word] for word in stemmed_query if word in vocabulary]]
            df_t=len(results)

            #Inverse document frequency
            #N=len(df)
            N=7200
            idf=math.log(N/(df_t+1))


            #TD-INF
            td_inf_i=tf* idf

            td_inf[word]=td_inf_i  #dictionary of td_idf for every word in the query

    dataset=pd.DataFrame([td_inf])
    return dataset

def query_ranking_allMatch(query_str,k,season=None):
    t=""
    for i in set(query_str.split()):
        t += i + " "
    query_str=t.strip()

    query_str=re.sub("\s{2,}"," ",query_str)
    
    parole=[word for word in stem_text(query_str)]
    check_parole=[word for word in stem_text(query_str) if word in vocabulary]
    if len(parole)== len(check_parole):
        

        indici=list(DF2[stem_text(query_str)][(DF2[stem_text(query_str)].T != 0).all()].index)




        #Inizialize a list for store the value of cosine_similarity
        list_cosine_similarity=[]

        #Define a new dataset. In this dataset we will add the cosine similarity.
        data=df[df.index.isin(indici)][['season', 'episodio', 'titolo', 'trama', 'guest_star', 'prima_visione']]


        #Define a new dataset. In this dataset we will put the tf-idf of the matching documents.
        a=DF2[DF2.index.isin(indici)][stem_text(query_str)]
        a=a.reindex(sorted(a.columns), axis=1) 

        #Evaluate tf-idf of query
        td_idf_query=querty_td_idf_allMatch(query_str)
        td_idf_query=td_idf_query.reindex(sorted(a.columns), axis=1) 



        #Evaluate cosine similarity between each document and the query.


        for i,row in a.iterrows():


            num=np.dot(td_idf_query.values.flatten(),row.values)
            den=np.linalg.norm(td_idf_query.values.flatten()) * np.linalg.norm(row.values)

            cosine=num/den
            list_cosine_similarity.append(cosine)

        data["Similarity"]= list_cosine_similarity
        dd=data.sort_values("Similarity",ascending=False)
        if season==None:
            dd=dd[['season', 'episodio', 'titolo', 'trama', 'guest_star', 'prima_visione']]
        else:
            dd=dd[['season', 'episodio', 'titolo', 'trama', 'guest_star', 'prima_visione']][df.season==season]
        dd.reset_index(drop=True,inplace=True)
        display(HTML(dd.head(k).to_html()))
        
        #return dd

    else:
        print("Impossibile trovare un documento che contenga tutta la parola nella query, controllare l'input per le parole errate.")
       # return 0

## Ranking engine
Vengono ritornati i primi k documenti più rilevanti

### Funzioni

In [22]:
def querty_td_idf(query_str):
    #Inizialize dataset
    td_inf={}
    
    #Remove double spaces
    query_str=re.sub("\s{2,}"," ",query_str)
    
    #Stem the query
    stemmed_query=stem_text(query_str)
    
    
    word_in_query=set(stemmed_query)
    
    
    for word in word_in_query:
        #DF document frequency
        df_t=vocabulary[word]
        if df_t==0:
            raise Exception("Can't find a document that contain all the word in the query, check input for misspelled words.")
           # raise Value Error("Can't find a document that contain all the word in the query, check input for misspelled words.")
            #print("Can't find a document that contain all the word in the query, check input for misspelled words.")
            #return None
        else:
    
            #TF
            ##occurence of t word in the query
            t_in_document=" ".join(stemmed_query).count(word) #count on total query or stemmed query? for now stemmed query


            ##Word in  the query
            parole_in_d=len(stemmed_query) #count on stemmed query or total query?for now stemmed query

            tf=t_in_document/parole_in_d
            
            #DF document frequency
            #df_t=vocabulary[word]
            results = [inverted_idx[str(item)] for item in [word_dict[word] for word in stemmed_query if word in vocabulary]]
            df_t=len(results)

            #Inverse document frequency
            #N=len(df)
            N=7200
            idf=math.log(N/(df_t+1))


            #TD-INF
            td_inf_i=tf* idf

            td_inf[word]=td_inf_i  #dictionary of td_idf for every word in the query

    dataset=pd.DataFrame([td_inf])
    return dataset

In [15]:
def query_ranking(query_str,k,season=None):
    t=""
    for i in set(query_str.split()):
        t += i + " "
    query_str=t.strip()

    query_str=re.sub("\s{2,}"," ",query_str)

    parole=[word for word in stem_text(query_str)]
    check_parole=[word for word in stem_text(query_str) if word in vocabulary]


    indici=list(DF2[stem_text(query_str)].index)




    #Inizialize a list for store the value of cosine_similarity
    list_cosine_similarity=[]



    #Define a new dataset. In this dataset we will add the cosine similarity.
    data=df[df.index.isin(indici)][['season', 'episodio', 'titolo', 'trama', 'guest_star', 'prima_visione']]


    #Define a new dataset. In this dataset we will put the tf-idf of the matching documents.
    a=DF2[DF2.index.isin(indici)][stem_text(query_str)]
    a=a.reindex(sorted(a.columns), axis=1) 

    #Evaluate tf-idf of query
    td_idf_query=querty_td_idf(query_str)
    td_idf_query=td_idf_query.reindex(sorted(a.columns), axis=1) 


    #Evaluate cosine similarity between each document and the query.


    for i,row in a.iterrows():

        num=np.dot(td_idf_query.values.flatten(),row.values)
        den=np.linalg.norm(td_idf_query.values.flatten()) * np.linalg.norm(row.values)

        cosine=num/den
        list_cosine_similarity.append(cosine)

    data["Similarity"]= list_cosine_similarity
    dd=data.sort_values("Similarity",ascending=False)
    #dd=dd[['season', 'episodio', 'titolo', 'trama', 'guest_star', 'prima_visione','Similarity']]
    if season==None:
        dd=dd[['season', 'episodio', 'titolo', 'trama', 'guest_star', 'prima_visione']]
    else:
        dd=dd[['season', 'episodio', 'titolo', 'trama', 'guest_star', 'prima_visione']][df.season==season]
    dd.reset_index(drop=True,inplace=True)
    display(HTML(dd.head(k).to_html()))

# Try the ranking engine all Match

In [28]:
k=5
query="luca paolo"
query_ranking_allMatch(query,k)

,season,episodio,titolo,trama,guest_star,prima_visione
0,6,45,Girano voci,"Le addette alle pulizie rivelano a Luca e Paolo che, secondo alcune voci, saranno trasferiti in Cina.",NaN,22 settembre 2017
1,1,355,Regressione,Luca prende una brutta botta in testa e regredisce fino all'età di 6 anni. Paolo è costretto a prendersene cura.,NaN,18 febbraio 2004
2,2,32,La moneta,"Luca e Paolo vogliono prendersi un caffè, ma sono senza monete.",['Bedy Moratti'],NaN
3,1,417,La scommessa,Luca e Paolo costringono Silvano a fare delle scommesse stupide con loro.,NaN,18 aprile 2004
4,5,143,La vedova,Luca e Paolo vogliono convincere la vedova di un dirigente defunto a lasciare loro parte dell'eredità.,['Anna Bonasso'],1º febbraio 2012


# Try the ranking engine

In [20]:
k=10
query="luca paolo geller silvano"
query_ranking(query,k,3)

/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_1672/1862020553.py:44: RuntimeWarning: invalid value encountered in scalar divide
  cosine=num/den
/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_1672/1862020553.py:53: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dd=dd[['season', 'episodio', 'titolo', 'trama', 'guest_star', 'prima_visione']][df.season==season]


,season,episodio,titolo,trama,guest_star,prima_visione
0,3,226,Parasole,"Luca trova un parasole in allegato al giornale e lo regala a Silvano, ma quando Paolo viene a conoscenza della cosa la prende male.",NaN,3 aprile 2006
1,3,39,Caramelle,"Luca e Paolo cercano invano di fare uno scherzo molto ""sporco"" a Silvano.",NaN,20 dicembre 2005
2,3,32,Gita aziendale,"Luca e Paolo sono stati esclusi dalla gita aziendale in montagna, ma Silvano non ha il coraggio di dirglielo.",NaN,20 dicembre 2005
3,3,256,Il funerale della mamma del Presidente,"Tutti i dipendenti partecipano al funerale della madre del Presidente con l'eccezione di Luca e Paolo, che lo disertano con gioia. Quando, però, i due scopriranno da Silvano che i dipendenti che hanno partecipato al funerale avranno la giornata libera, i due fingeranno di essere stati presenti.",NaN,15 aprile 2006
4,3,118,Il primo giorno di Patti,"Con un salto indietro nel tempo, scopriamo l'arrivo in azienda di Patti: assunta come segretaria di Ilaria, Patti viene immediatamente presa di mira da Luca e Paolo, che cercano subito di farle fare brutta figura con il suo nuovo capo, ma con Silvano è subito un colpo di fulmine",NaN,20 febbraio 2006
5,3,57,Un piano pauroso,Luca e Paolo spaventano a morte Silvano con una misteriosa VHS[6],NaN,20 gennaio 2006
6,3,55,Lo scherzo dell'uovo,Luca e Paolo mostrano a Silvano lo scherzo dell'uovo sodo.,NaN,20 gennaio 2006
7,3,205,Prurito,Silvano parte per le ferie e si vendica di uno scherzo subìto da Luca e Paolo.,['Igor Loddo'],29 marzo 2006
8,3,49,Lo scherzone,Silvano fa uno scherzo a Luca e Paolo per vendicarsi.,['Antonio Zavatteri'],20 gennaio 2006
9,3,155,Pericolo tattoo,Paolo e Luca fanno un tatuaggio a Silvano.,NaN,12 marzo 2006


# Prove

In [ ]:
k=5
query="lotteria"
query_ranking_allMatch(query,k)
query_ranking(query,k)